In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import math
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import os
from tqdm import tqdm
np.random.seed(7)

Using TensorFlow backend.
/home/alex/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data_path = '/home/alex/Desktop/kaggle/Grocery/data/'
Holiday_Events = pd.read_csv(data_path + 'holidays_events.csv')
Items = pd.read_csv(data_path + 'items.csv')
Oil = pd.read_csv(data_path + 'oil.csv')
Sample_Submission = pd.read_csv(data_path + 'sample_submission.csv')
Stores = pd.read_csv(data_path + 'stores.csv')
Test = pd.read_csv(data_path + 'test.csv')
Transactions = pd.read_csv(data_path + 'transactions.csv')
database_path = data_path + 'databases2/'

In [3]:
Test.head()

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


In [4]:
Test_Items = Test.loc[:,['item_nbr','store_nbr']]
#Test_Items = Test_Items.groupby(['id','store_nbr']).size().reset_index().rename(columns={0:'count'})
#Test_Items.head()
Test_Items = Test_Items.drop_duplicates()
print(Test_Items.shape)

(210654, 2)


In [5]:
#Check for existence of item-store combinations within train data

Train_Items = []
for root, dirs, files in os.walk(database_path):
    for filename in files:
        Train_Items.append(filename)
        


In [47]:
T2 =[T[5:] for T in Train_Items]
T3 = [T[0:len(T2[1])-4] for T in T2]

In [6]:
train_item_set.head()

,item_nbr
0,103665.0
1,105574.0
2,105575.0
3,108079.0
4,108701.0


257it [00:30,  8.56it/s]

In [7]:
train_item_set = pd.DataFrame({'item_nbr' : []})
Train_Items = pd.read_csv(data_path + 'train.csv', chunksize = 100000)
for a in tqdm(Train_Items):
    tmp = pd.DataFrame({'item_nbr': a.item_nbr.unique()})
    train_item_set = train_item_set.append(tmp)
#train_item_set.item_nbr = train_item_set.item_nbr.unique()

1255it [01:03, 19.78it/s]


In [8]:
train_item_set_list = [int(R) for R in train_item_set.item_nbr.tolist()]

In [11]:
train_item_set_list = train_item_set.item_nbr.tolist()
Test_Items['Candidate'] = Test_Items['item_nbr'].isin(train_item_set_list)

In [12]:
Test_Items.Candidate.value_counts()

True     207414
False      3240
Name: Candidate, dtype: int64

In [20]:
Test_Item_Candidates = Test_Items[Test_Items.Candidate == True]
Test_Item_Candidates.item_nbr[0]

96995

In [29]:
#Select a single item that is known to be a candidate
#Candidate 50 seems to have enough going on
Set_1 = pd.read_csv(database_path +'item_'+ str(Test_Item_Candidates.item_nbr[50])  +'.csv')
Set_1 = Set_1.drop(['Unnamed: 0'],axis=1)
Set_1.tail(17)

,time_step,unit_sales_1,unit_sales_2,unit_sales_3,unit_sales_4,unit_sales_5,unit_sales_6,unit_sales_7,unit_sales_8,unit_sales_9,...,onpromotion_44,onpromotion_45,onpromotion_46,onpromotion_47,onpromotion_48,onpromotion_49,onpromotion_50,onpromotion_51,onpromotion_52,onpromotion_53
1671,1672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1672,1673,6.0,5.0,2.0,-100.0,1.0,-100.0,2.0,9.0,-100.0,...,False,False,False,False,False,False,False,False,False,False
1673,1674,1.0,2.0,2.0,3.0,1.0,5.0,8.0,5.0,-100.0,...,False,False,False,False,False,False,False,False,False,False
1674,1675,5.0,3.0,13.0,7.0,-100.0,2.0,-100.0,3.0,-100.0,...,False,False,False,False,False,False,False,False,False,False
1675,1676,17.0,6.0,11.0,-100.0,3.0,13.0,-100.0,2.0,-100.0,...,False,False,False,False,False,False,False,False,False,False
1676,1677,2.0,22.0,13.0,6.0,5.0,1.0,1.0,15.0,-100.0,...,False,False,False,False,False,False,False,False,False,False
1677,1678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1678,1679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1679,1680,2.0,1.0,5.0,-100.0,-100.0,6.0,-100.0,2.0,-100.0,...,False,False,False,False,False,False,False,False,False,False
1680,1681,4.0,3.0,6.0,3.0,4.0,8.0,5.0,1.0,-100.0,...,False,False,False,False,False,False,False,False,False,False


In [78]:
def prep_data(dataset, target_Col, look_Back):
    X=[]
    y=[]
    for i in range(len(dataset) - look_Back - 16+1):
        X.append(dataset[i:i+look_Back,:])
        y.append(dataset[i + look_Back :i+ look_Back + 16,1])
        pred = dataset[len(dataset) - look_Back:len(dataset),:]
    return X, y, pred

In [79]:
t = pd.Series(range(1,10))
t[len(t) - 2:len(t)]

7    8
8    9
dtype: int64

In [80]:
aSet = Set_1.values
A, b, pred = prep_data(aSet, 1,30)

In [81]:
np.array(pred).shape

(30, 107)

In [36]:
A[1642][29]

array([1672, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False], dtype=object)

In [37]:
b[1642]

array([6.0, 1.0, 5.0, 17.0, 2.0, 0.0, 0.0, 2.0, 4.0, 1.0, 10.0, 2.0, 0.0,
       0.0, 8.0, -100.0], dtype=object)

In [82]:
np.array(b).shape
np.array(A).shape

(1643, 30, 107)

In [83]:
Set_1.columns[[1,2,54,55]]

Index(['unit_sales_1', 'unit_sales_2', 'onpromotion_1', 'onpromotion_2'], dtype='object')

In [92]:
X = np.array(A)[:,:,[1,2,54,55]]
pred = np.array(pred)[:,[1,2,54,55]]
y = np.array(b)
X.shape

(1643, 30, 4)

In [93]:
model = Sequential()
model.add(LSTM(20, input_shape = (30,4)))
model.add(Dense(16))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.fit(X, y, validation_split = .3, epochs = 100, batch_size = 100, verbose = 1)

Train on 1150 samples, validate on 493 samples
Epoch 1/100
1150/1150 [==============================] - 1s 601us/step - loss: 1111.2431 - val_loss: 984.3588
Epoch 2/100
1150/1150 [==============================] - 0s 188us/step - loss: 1109.7499 - val_loss: 983.3362
Epoch 3/100
1150/1150 [==============================] - 0s 185us/step - loss: 1108.2723 - val_loss: 982.1881
Epoch 4/100
1150/1150 [==============================] - 0s 177us/step - loss: 1106.5228 - val_loss: 980.7293
Epoch 5/100
1150/1150 [==============================] - 0s 195us/step - loss: 1104.3631 - val_loss: 978.7970
Epoch 6/100
1150/1150 [==============================] - 0s 181us/step - loss: 1101.5787 - val_loss: 976.3805
Epoch 7/100
1150/1150 [==============================] - 0s 184us/step - loss: 1098.0859 - val_loss: 973.7685
Epoch 8/100
1150/1150 [==============================] - 0s 184us/step - loss: 1094.3076 - val_loss: 971.0000
Epoch 9/100
1150/1150 [==============================] - 0s 185us/step - 

1150/1150 [==============================] - 0s 186us/step - loss: 1039.9990 - val_loss: 945.6933
Epoch 75/100
1150/1150 [==============================] - 0s 174us/step - loss: 1039.2677 - val_loss: 949.0173
Epoch 76/100
1150/1150 [==============================] - 0s 175us/step - loss: 1038.6931 - val_loss: 948.9347
Epoch 77/100
1150/1150 [==============================] - 0s 179us/step - loss: 1037.5617 - val_loss: 949.2906
Epoch 78/100
1150/1150 [==============================] - 0s 177us/step - loss: 1037.0450 - val_loss: 948.0350
Epoch 79/100
1150/1150 [==============================] - 0s 175us/step - loss: 1036.1458 - val_loss: 950.5244
Epoch 80/100
1150/1150 [==============================] - 0s 174us/step - loss: 1035.5316 - val_loss: 949.9902
Epoch 81/100
1150/1150 [==============================] - 0s 177us/step - loss: 1034.0519 - val_loss: 949.8757
Epoch 82/100
1150/1150 [==============================] - 0s 180us/step - loss: 1032.8192 - val_loss: 951.6557
Epoch 83/100
1

In [94]:
X.shape

(1643, 30, 4)

In [95]:
s = pred.reshape(1,pred.shape[0],pred.shape[1])

In [96]:
model.predict(s)

array([[-5.31099796, -5.80421019, -6.36868286, -6.52695274, -6.23614216,
        -5.77478218, -6.40527439, -6.49669313, -7.19526625, -7.21782923,
        -6.88676262, -6.89384747, -7.2648654 , -6.0042634 , -6.56038666,
        -6.48779869]], dtype=float32)